In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Libarires

In [ ]:
!pip install catboost

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Evaluation Matrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

## NLTK
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt') # download the necessary NLTK packages
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Load Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dream/dream_reports - TextBlob - 5 Labels.csv',encoding = 'latin-1')

In [ ]:
df.head(3)

,dream_report,sentences,sentiment_score,sentiment_label
0,"The one at the Meads's house, where it's bigge...","[""The one at the Meads's house, where it's big...","[0.025992063492063486, -0.07668269230769231, 0...","['very negative', 'negative', 'neutral', 'neut..."
1,I'm at a family reunion in a large fine house ...,"[""I'm at a family reunion in a large fine hous...","[0.20773809523809522, -0.021875000000000006, -...","['very negative', 'negative', 'very negative',..."
2,I watch a plane fly past and shortly realize i...,['I watch a plane fly past and shortly realize...,"[0.1375, 0.027777777777777776, 0.0, 0.0, -0.2,...","['very negative', 'very negative', 'neutral', ..."


## Take sentences and Label

In [ ]:
sentences = df['sentences'].values
labels = df['sentiment_label'].values

## Create New Dataset

In [ ]:
def convert_to_list(input_string):
    return eval(input_string)

label = []
for lab in labels:
  lab = convert_to_list(lab)
  for i in range(len(lab)):
    label.append(lab[i])

sentence = []
for sent in sentences:
  sent = convert_to_list(sent)
  for i in sent:
    sentence.append(i)

## Final Dataframe

In [ ]:
data = pd.DataFrame()
data['sentences'] = sentence
data['sentimentlabel'] = label

## Clean Data

In [ ]:
def clean_text(text):
    # remove numbers
    text = ''.join([i for i in text if not i.isdigit()])
    
    # remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # tokenize the text
    tokens = word_tokenize(text)
    
    # lowercase the tokens
    tokens = [word.lower() for word in tokens]
    
    # remove stop words
    tokens = [word for word in tokens if word not in stop_words]
    
    # lemmatize the tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # join the tokens back into a single string
    return " ".join(tokens)

In [ ]:
data['sentence_clean'] = data['sentences'].apply(lambda x: clean_text(x))

In [ ]:
data

,sentences,sentimentlabel,sentence_clean
0,"The one at the Meads's house, where it's bigge...",3,one mead house bigger inside there european vi...
1,"Now outside, I'm waiting for my aunt to pick m...",0,outside im waiting aunt pick arrives little ro...
2,Inside (?),1,inside
3,"I sit with a couple of people, playing with a ...",1,sit couple people playing string blue balloon
4,I'm at a family reunion in a large fine house ...,3,im family reunion large fine house ground mayb...
...,...,...,...
325277,What would they think of me now?,1,would think
325278,Would they hate me?,3,would hate
325279,They loved me once.,2,loved
325280,In real life I know this would never have happ...,3,real life know would never happened


## Convert Cat to Int

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder() ## Encoder that convert and store all the information
data['sentimentlabel'] = lb.fit_transform(data['sentimentlabel'])

mapping = dict(zip(lb.classes_, range(len(lb.classes_))))
mapping

{'negative': 0,
 'neutral': 1,
 'positive': 2,
 'very negative': 3,
 'very positive': 4}

## Bert

In [ ]:
!pip install transformers

In [ ]:
!pip install tensorflow_text

In [ ]:
!pip install tokenization

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

In [ ]:
!pip install pytorch-transformers

In [ ]:
try:
    %tensorflow_version 2.x
except Exception as ex:
    print(ex)
    pass
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import re
import unicodedata
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
from transformers import *
import bert
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig

##Loading DistilBERT Tokenizer and the DistilBERT model


In [ ]:
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

## Preparing input for the model


In [ ]:
max_len=260
sentences=data['sentence_clean'] 
labels=data['sentimentlabel']
len(sentences),len(labels)

## Let's take a sentence from the dataset and understand the input and output of the DistilBERT


In [ ]:
dbert_tokenizer.tokenize(sentences[0])

## Input ids and the attention masks from the tokenizer

In [ ]:
dbert_inp=dbert_tokenizer.encode_plus(sentences[0],add_special_tokens = True,max_length =20,pad_to_max_length = True,truncation=True)
dbert_inp

## NN model using DistilBERT embeddings

In [ ]:
num_classes=len(df.label_text.unique())

In [ ]:
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout= Dropout(0.5)(dense)
    pred = Dense(num_classes, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

In [ ]:
model=create_model()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 260)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 260)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 260, 768),                                                   
                                 hidden_states=None                                           

## Input Model

In [ ]:
input_ids=[]
attention_masks=[]

for sent in df['text']:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

In [ ]:
input_ids=[]
attention_masks=[]

for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
len(input_ids),len(attention_masks),len(labels)

(65929, 65929, 65929)

## Save the model input in the pickle files to use it later without performing the above steps

In [ ]:
print('Preparing the pickle file.....')

pickle_inp_path='./dbert_inp.pkl'
pickle_mask_path='./dbert_mask.pkl'
pickle_label_path='./dbert_label.pkl'

pickle.dump((input_ids),open(pickle_inp_path,'wb'))
pickle.dump((attention_masks),open(pickle_mask_path,'wb'))
pickle.dump((labels),open(pickle_label_path,'wb'))


print('Pickle files saved as ',pickle_inp_path,pickle_mask_path,pickle_label_path)

Preparing the pickle file.....
Pickle files saved as  ./dbert_inp.pkl ./dbert_mask.pkl ./dbert_label.pkl


In [ ]:
print('Loading the saved pickle files..')

input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))

print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))

Loading the saved pickle files..
Input shape (65929, 260) Attention mask shape (65929, 260) Input label shape (65929,)


## label

In [ ]:
label_class_dict={0:'Not Related to HIV',1:'Related to HIV'}
target_names=label_class_dict.values()

## Compile Model

In [ ]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))


log_dir='dbert_model'
model_save_path='./dbert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model.compile(loss=loss,optimizer=optimizer, metrics=[metric])

Train inp shape (52743, 260) Val input shape (13186, 260)
Train label shape (52743,) Val label shape (13186,)
Train attention mask shape (52743, 260) Val attention mask shape (13186, 260)


In [ ]:
callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
model.compile(loss=loss,optimizer=optimizer, metrics=[metric])

## Train Model

In [ ]:
history=model.fit([train_inp,train_mask],train_label,batch_size=16,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

In [ ]:
trained_model = create_model()
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights(model_save_path)

In [ ]:
preds = trained_model.predict([val_inp,val_mask],batch_size=16)
pred_labels = preds.argmax(axis=1)

In [ ]:
# Creating the confusion matrix:
e_cm = confusion_matrix(val_label, pred_labels)
# Visualization:
f, ax = plt.subplots(figsize=(8,8))
sns.heatmap(e_cm, annot=True, linewidth=0.7, linecolor='blue', fmt='.0f', ax=ax, cmap='Blues')
plt.title('Bert Classification Confusion Matrix TFIDF')
plt.xlabel('y_pred')
plt.ylabel('y_test')
plt.show()

In [ ]:
print(classification_report(val_label, pred_labels))